# Computer Vision Business Recommendations
#### Luis Estrada, Michael Eugene, Maddie Hince, Kilmar Lazo, Cat Murad, and Elijah Soba

## Overview

## Business Understanding

## Data Understanding

### Data Sources
We used data from three main sources to make recommendations for Computer Vision:
- Box Office Mojo
- The Numbers
- IMDB (Internet Movie Database)

### Description of Data
The data from Box Office Mojo included movie domestic and foreign gross, and movie production studio.
- Dataset contained in a .csv file
- Approximately 39% missing data in foreign gross column

Data from The Numbers included production budget and worldwide gross information.
- Dataset contained in a .csv file
- Approximately 6% missing data in worldwide gross

Data from IMDB included general movie information such as title, average ratings, and genres.
- Data represented as a relational SQL database
- Most analysis done using __movie_basics__ and __movie_ratings__ tables

We only considered movies in each dataset that had been released in 2010 or later.

## Data Analysis

### Highest-Rated Genres

### Budget-to-Box Office Ratio

### Highest-Grossing Studios

## Statistical Communication

After identifying __animation__ as the genre Computer Vision should pursue, we wanted to explore the difference between the genre's foreign and domestic gross. 

We hypothesized that:
- A movie in the animation category will gross higher internationally than domestically. 

Based on the results of our t-test, we can determine where to allocate more marketing efforts to maximize profitability.

### T-test
We want to know if __animation__ movies gross more internationally or domestically. 
- We chose the __animation__ genre because it is one of the genres with the highest average ratings, and appears as a genre in highly-grossing films from the highest-grossing studio (BV Studios, which turns out to be Disney/Pixar).

Our __null hypothesis__ is that the animation genre grosses the same internationally and domestically.

Our __alternative__ hypothesis is that animation movies gross more internationally than domestically. 

The results of our statistical test show that with 95% confidence we can conclude foreign gross is significantly greater than domestic gross regarding movies with the __animation__ genre. 
- This indicates that more money should be spent marketing to foreign markets as opposed to domestic.

## Conclusion